In [1]:
import pickle 
import gzip

submissions = pickle.load(gzip.open("../../Results/submissions.pickle.gz","rb"))
comments = pickle.load(gzip.open("../../Results/comments.pickle.gz","rb"))
# load dict storing list of moderators and bots
ignore = pickle.load(gzip.open("../../Results/ignore.pickle.gz","rb"))

In [17]:
num_posts = {}

for subreddit in ['politics','The_Donald','SandersForPresident','Conservative']:
    num_comments = []
    num_submissions = []
    
    for commentator in comments[subreddit]['commentators']:
        if commentator not in ignore:
            num_comments.append(len(comments[subreddit]['comments'][commentator]['votes']))
    for submissioner in submissions[subreddit]['submissioners']:
        if submissioner not in ignore:
            num_submissions.append(len(submissions[subreddit]['submissions'][submissioner]['votes']))
    
    num_posts[subreddit] = {
        'comments': num_comments,
        'submissions': num_submissions,
    }

pickle_out = open("../../Results/num_posts.pickle","wb")
pickle.dump(num_posts, pickle_out)
pickle_out.close()

In [18]:
len(num_posts['The_Donald']['submissions']),sum(num_posts['The_Donald']['submissions'])

(221408, 2787259)

In [19]:
len(num_posts['politics']['submissions']),sum(num_posts['politics']['submissions'])

(125735, 830291)

In [24]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
plotly.tools.set_credentials_file(username='soli__', api_key='azOUWrgxv12Fmya1i5W7')

def plot_sample(vals,title,x_label,y_label,activity,sample_size=50000,log=True):
    data = []
    
    subreddits = ['SandersForPresident','politics','Conservative','The_Donald']
    for subreddit in subreddits:
        if len(vals[subreddit][activity]) < sample_size:
            subreddits.remove(subreddit)
            
    colors = {
        'The_Donald': '(139,0,0)',
        'Conservative': '(255,0,0)',
        'Republican': '(205,92,92)',
        'NeutralPolitics': '(112,128,144)',
        'politics': '(0,191,255)',
        'democrats': '(135,206,250)',
        'progressive': '(0,0,139)',
        'SandersForPresident': '(0,0,255)',
    }
    
    for subreddit in subreddits:
       
        X = []
        Y = []
        
        sample = np.random.choice(vals[subreddit][activity], sample_size)
        for key, val in enumerate(sorted(sample,reverse=True)):
            if log:
                X.append(np.log(key+1))
                if val <= 0:
                    Y.append(0)
                else:
                    Y.append(np.log(val))
            else:
                X.append((key+1))
                if val <= 0:
                    Y.append(0)
                else:
                    Y.append((val))

        trace = go.Scatter(
            x = X,
            y = Y,
            mode = 'lines',
            name = subreddit,
            line = {
                'color' : ('rgb' + colors[subreddit]),
            }
        )

        data.append(trace)
        
    title =  {
        'text':'<b>' + title + '</b>',
        'font': {
            'color': 'black',
            'size': 30,
            
        }
    }
        
        
    layout = go.Layout(
            title = title,
            xaxis = dict(
                title = x_label,
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='black'
                )
            ),
            yaxis = dict(
                title = y_label,
                titlefont = dict(
                    family='Courier New, monospace',
                    size=18,
                    color='black'
                )
            ),
         legend = {
                'font': {
                    'size': 15
                }
            },
        )
    
    fig = go.Figure(data=data, layout=layout)
    
    display(py.iplot(fig, filename='line-mode'))

In [28]:
plot_sample(num_posts,'title','x_label','y_label','submissions',sample_size=200000,log=True)

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~soli__/0 or inside your plot.ly account where it is named 'line-mode'
